In [50]:
import torch

梯度指向值变化最大的方向
比如方向（2，4）与等高线正交


In [51]:
#传统按位计算向量点积和numpy优化计算向量点积
import numpy as np
import time 
a = np.random.rand(1000000)
b = np.random.rand(1000000)
tic = time.time()
c = np.dot(a, b)
toc = time.time()

print(c)
print("Vectorized version:" + str(1000 * (toc - tic)) + 'ms')

c = 0
tic = time.time()
for i in range(1000000):
    c += a[i] * b[i]
toc = time.time()
print(c)
print("For loop:" + str(1000 * (toc - tic)) + 'ms')


250273.97645649494
Vectorized version:0.0ms
250273.97645649363
For loop:495.30744552612305ms


反向累积总结：
构造计算图
前向：执行图，存储中间的结果
反向：从相反方向执行图
理解：
正向：求复合函数各层的表达式
反向：求偏导数和梯度

因为要存储正向累积的所有结果，所以神经网络会很消耗内存

In [52]:
import torch 
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [53]:
#在计算y关于x的梯度之前，需要一个地方来存储梯度
x.requires_grad_(True)#等价于x.torch.arange(4.0, )
x.grad
#注意，只有浮点数float才能求梯度


In [54]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [55]:
#调用反向传播函数自动计算y关于x每个分量的梯度
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [56]:
x.grad == 4 * x


tensor([True, True, True, True])

In [57]:
#在默认情况下，pytorch会累积（理解累计含义）梯度，需要清除之前的值
x.grad.zero_()#末尾下划线表示，将x的每个梯度分量设为0
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [58]:
# dl中，目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和
x.grad.zero_()
y = x * x
#对非标量调用backward需要传入一个gradient参数
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [59]:
x.grad.zero_()
y = x * x
u = y.detach()#表示u是y解除与x函数关系的常数
z = u * x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [60]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [62]:
#范数用于计算向量的模长
#b = b.norm()
#即使构建函数的计算图需要通过python控制流（条件，循环，迭代，控制流或任意函数调用）
#我们依然可以调用backward得到变量的梯度
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
        if b.sum() > 0:
            c = b
        else:
            c = 100 * b
        return c

a = torch.randn(size=(), requires_grad= True)#size=0 表示张量，randn为随机生成数
d = f(a)
d.backward()

a.grad == d / a

tensor(True)